In [ ]:
from cosmoprimo import fiducial
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
cosmo_desi = fiducial.DESI()

tracers = ['lrg', 'elg', 'qso']
redshifts = [0.8, 1.1, 1.4]

cosmo_list = []
cosmo_names = ['000', '001', '002', '003', '004']
for cosmo_name in cosmo_names:
    cosmo = fiducial.AbacusSummit(name=cosmo_name, engine='class')
    cosmo_list.append(cosmo)

In [ ]:
res = []
alpha_values = {}
for tracer, redshift in zip(tracers, redshifts):
    alpha_values[tracer] = {}
    for j in [0, 3, 4]:
        cosmo_true = cosmo_list[j]
        cosmo_true_name = cosmo_names[j]
        rs_true = cosmo_true.rs_drag
        h_true = cosmo_true.efunc(redshift).item()
        da_true = cosmo_true.angular_diameter_distance(redshift).item()
    
        for cosmo_grid, cosmo_grid_name in zip(cosmo_list, cosmo_names):
            for cosmo_temp, cosmo_temp_name in zip(cosmo_list, cosmo_names):
                rs_temp = cosmo_temp.rs_drag
                h_cat = cosmo_grid.efunc(redshift).item()
                da_cat = cosmo_grid.angular_diameter_distance(redshift).item()
                q_par = h_cat / h_true
                q_perp = da_true / da_cat
                rs_ratio = rs_temp / rs_true
                
                res.append([tracer, cosmo_true_name, cosmo_grid_name, cosmo_temp_name, q_par, q_perp, rs_ratio])
df = pd.DataFrame(res, columns=['tracer', 'cosmo_true', 'cosmo_grid', 'cosmo_temp', 'q_par', 'q_perp', 'rs_ratio'])
df['alpha_par'] = df['q_par'] * df['rs_ratio']
df['alpha_perp'] = df['q_perp'] * df['rs_ratio']
df = df.set_index(['tracer', 'cosmo_true', 'cosmo_grid', 'cosmo_temp'])
df = df.sort_index()

In [ ]:
for tracer in tracers:
    df.loc[tracer].to_csv(f'alphas_{tracer}_cubicbox.csv')

In [ ]:
alpha_values = {}
for tracer, redshift in zip(tracers, redshifts):
    alpha_values[tracer] = {}
    for j in [0, 3, 4]:
        cosmo_true = cosmo_list[j]
        cosmo_true_name = cosmo_names[j]
        rs_true = cosmo_true.rs_drag
        h_true = cosmo_true.efunc(redshift).item()
        da_true = cosmo_true.angular_diameter_distance(redshift).item()
        
        alpha_values[tracer][cosmo_true_name] = {}
        values = alpha_values[tracer][cosmo_true_name]
        for cosmo_grid, cosmo_grid_name in zip(cosmo_list, cosmo_names):
            values[cosmo_grid_name] = {}
            for cosmo_temp, cosmo_temp_name in zip(cosmo_list, cosmo_names):
                values[cosmo_grid_name][cosmo_temp_name] = {}
                rs_temp = cosmo_temp.rs_drag
                h_cat = cosmo_grid.efunc(redshift).item()
                da_cat = cosmo_grid.angular_diameter_distance(redshift).item()
                q_par = h_cat / h_true
                q_perp = da_true / da_cat
                rs_ratio = rs_temp / rs_true
                alpha_par = q_par * rs_ratio
                alpha_perp = q_perp * rs_ratio
                values[cosmo_grid_name][cosmo_temp_name]['q_par'] = q_par
                values[cosmo_grid_name][cosmo_temp_name]['q_perp'] = q_perp
                values[cosmo_grid_name][cosmo_temp_name]['rs_ratio'] = rs_ratio
                values[cosmo_grid_name][cosmo_temp_name]['alpha_par'] = alpha_par
                values[cosmo_grid_name][cosmo_temp_name]['alpha_perp'] = alpha_perp

In [ ]:
alpha_values['lrg']['000']['000']['002']